In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install face_recognition opencv-python

In [ ]:
import cv2
import os
import face_recognition
from tqdm import tqdm

def extract_face_frames_from_videos(video_folder, output_folder, label, max_frames_per_video=10):
    os.makedirs(os.path.join(output_folder, label), exist_ok=True)
    video_files = [f for f in os.listdir(video_folder) if f.endswith('.mp4')]

    for video_file in tqdm(video_files, desc=f"Processing {label} videos"):
        video_path = os.path.join(video_folder, video_file)
        cap = cv2.VideoCapture(video_path)

        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        interval = max(1, frame_count // max_frames_per_video)

        saved = 0
        frame_no = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or saved >= max_frames_per_video:
                break

            if frame_no % interval == 0:
                rgb_frame = frame[:, :, ::-1]
                faces = face_recognition.face_locations(rgb_frame)

                if faces:
                    top, right, bottom, left = faces[0]  # Take first face
                    face_img = rgb_frame[top:bottom, left:right]
                    face_img = cv2.resize(face_img, (224, 224))

                    save_path = os.path.join(output_folder, label, f"{video_file[:-4]}_frame{saved}.jpg")
                    cv2.imwrite(save_path, cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR))
                    saved += 1

            frame_no += 1

        cap.release()


In [ ]:
extract_face_frames_from_videos('/content/drive/MyDrive/FF/real', '/content/drive/MyDrive/output', 'real')


Processing real videos: 100%|██████████| 200/200 [1:03:25<00:00, 19.03s/it]


In [ ]:
extract_face_frames_from_videos('/content/drive/MyDrive/FF/fake', '/content/drive/MyDrive/output', 'fake')

Processing fake videos: 100%|██████████| 200/200 [59:04<00:00, 17.72s/it]
